## model architecture

In [6]:
from transformers import BertModel, BertForSequenceClassification

In [7]:
model_name = 'bert-base-uncased'

In [8]:
model = BertModel.from_pretrained(model_name)
cls_model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
cls_model
# 其实与bert基本上一样，只是在末尾接了一个输入是768，输出是2的分类
#   (classifier): Linear(in_features=768, out_features=2, bias=True)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### summary
- bert: encoder of transformer(+embedding, pooler)  
    - transformer: encoder-decoder
- bert
    - embeddings
    - encoder(12layers)
          - self-attention（qkv）
          - feed forward
    - pooler

## 参数量统计

In [28]:
# 参数量统计
total_params = 0
total_learnable_params = 0
total_embedding_params = 0
total_encoder_params = 0
total_pooler_params = 0

for name, param in model.named_parameters():
    print(name, '->', param.shape, '->', param.numel())
    if 'embedding' in name:
        total_embedding_params += param.numel()
    if 'encoder' in name:
        total_encoder_params += param.numel()
    if 'pooler' in name:
        total_pooler_params += param.numel()
    if param.requires_grad:
        total_learnable_params += param.numel()
    total_params += param.numel()

embeddings.word_embeddings.weight -> torch.Size([30522, 768]) -> 23440896
embeddings.position_embeddings.weight -> torch.Size([512, 768]) -> 393216
embeddings.token_type_embeddings.weight -> torch.Size([2, 768]) -> 1536
embeddings.LayerNorm.weight -> torch.Size([768]) -> 768
embeddings.LayerNorm.bias -> torch.Size([768]) -> 768
encoder.layer.0.attention.self.query.weight -> torch.Size([768, 768]) -> 589824
encoder.layer.0.attention.self.query.bias -> torch.Size([768]) -> 768
encoder.layer.0.attention.self.key.weight -> torch.Size([768, 768]) -> 589824
encoder.layer.0.attention.self.key.bias -> torch.Size([768]) -> 768
encoder.layer.0.attention.self.value.weight -> torch.Size([768, 768]) -> 589824
encoder.layer.0.attention.self.value.bias -> torch.Size([768]) -> 768
encoder.layer.0.attention.output.dense.weight -> torch.Size([768, 768]) -> 589824
encoder.layer.0.attention.output.dense.bias -> torch.Size([768]) -> 768
encoder.layer.0.attention.output.LayerNorm.weight -> torch.Size([768])

In [23]:
total_learnable_params

109482240

In [24]:
total_params

109482240

In [27]:
params = [total_embedding_params, total_encoder_params, total_pooler_params]
for param in params:
    print(param/sum(params))

0.21772649152958506
0.776879099295009
0.005394409175405983
